# Función objetivo para optimización con Hyperopt

In [16]:
def objective_rf(params):
    # Convertir los parámetros a enteros donde sea necesario
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])


    model = RandomForestRegressor(**params, criterion="squared_error",
                                  n_jobs=-1, random_state=seed)

    est = int(params['n_estimators'])
    # lg_rt = params['learning_rate']
    max_dep = int(params['max_depth'])
    min_s_s = int(params['min_samples_split'])
    min_s_l = int(params['min_samples_leaf'])
    # subs = params['subsample']
    # col_by = params['colsample_bytree']
    # gam = params['gamma']
    # alp = params['reg_alpha']
    # lamb = params['reg_lambda']


    # kfold = KFold(n_splits=10, shuffle=True, random_state=seed)


    # score = -np.mean(cross_val_score(model, X_train, y_train, cv=kfold,
    #                                 scoring='r2', n_jobs=-1))

    # print("Score CV RandomForestRegressor: ", score)

    # return {'loss': score, 'status': STATUS_OK}

    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

    # Evaluar usando cross_validate para obtener RMSE, R² y MAE
    cv_results = cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring, return_train_score=False)

    # Calcular la media de cada métrica
    #mean_rmse = -np.mean(cv_results['test_rmse'])  # Negativo porque Hyperopt minimiza
    mean_rmse = np.mean(cv_results['test_rmse'])  # Negativo porque Hyperopt minimiza
    mean_r2 = np.mean(cv_results['test_r2'])
    mean_r2_n = -np.mean(cv_results['test_r2'])
    mean_mae = np.mean(cv_results['test_mae'])

    # print(f"RMSE: {mean_rmse}, R2: {mean_r2}, MAE: {mean_mae}")


    print(f"RMSE: {mean_rmse}, R² : {mean_r2}, MAE: {mean_mae} ;\n\t PARAMS; estimator: {est}, max_depth: {max_dep}, min_samples_split: {min_s_s}, min_samples_leaf: {min_s_l}")
    print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    return {'loss': mean_r2_n, 'status': STATUS_OK, 'rmse': mean_rmse, 'r2': mean_r2, 'mae': mean_mae}


space_rf = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 1500, 2500, 100)),
    'max_depth': scope.int(hp.quniform('max_depth', 15, 30, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 10, 1)),
}


def objective_svr(params):
    model = SVR(**params, kernel='rbf', gamma='scale', max_iter=-1)
    #model = SVR(**params, kernel='rbf')

    est = int(params['C'])
    lg_rt = params['epsilon']
    max_dep = int(params['degree'])
    # min_s_s = int(params['min_samples_split'])
    # min_s_l = int(params['min_samples_leaf'])

    # kfold = KFold(n_splits=10, shuffle=True, random_state=seed)


    # score = -np.mean(cross_val_score(model, X_train, y_train, cv=kfold,
    #                                 scoring='r2', n_jobs=-1))

    # print("Score CV SVR: ", score)

    # return {'loss': score, 'status': STATUS_OK}

    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

    # Evaluar usando cross_validate para obtener RMSE, R² y MAE
    cv_results = cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring, return_train_score=False)

    # Calcular la media de cada métrica
    #mean_rmse = -np.mean(cv_results['test_rmse'])  # Negativo porque Hyperopt minimiza
    mean_rmse = np.mean(cv_results['test_rmse'])  # Negativo porque Hyperopt minimiza
    mean_r2 = np.mean(cv_results['test_r2'])
    mean_r2_n = -np.mean(cv_results['test_r2'])
    mean_mae = np.mean(cv_results['test_mae'])

    # print(f"RMSE: {mean_rmse}, R2: {mean_r2}, MAE: {mean_mae}")
    print(f"RMSE: {mean_rmse}, R² : {mean_r2}, MAE: {mean_mae} ;\n\t PARAMS; C: {est}, epsilon: {lg_rt}, degree: {max_dep}")
    print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    return {'loss': mean_r2_n, 'status': STATUS_OK, 'rmse': mean_rmse, 'r2': mean_r2, 'mae': mean_mae}


space_svr = {
    #'C': hp.guniform('C', 100, 1000, 100),
    'C': hp.quniform('C', 1000, 3000, 100),
    'epsilon': hp.loguniform('epsilon', np.log(0.001), np.log(0.003)),
    'degree': scope.int(hp.quniform('degree', 4, 10, 1)),

}

In [17]:
# Optimización para cada modelo

def opt_bayesian(space_bayesian):
    print(f"\t Optimización de hiperparametros para Bayesian.....\n")
    start_time = time.time()

    trials_bayesian = Trials()
    best_params_bayesian = fmin(fn=objective_bayesian, space=space_bayesian, algo=tpe.suggest,
                                max_evals=100, trials=trials_bayesian)

    end_time = time.time()
    total_time = end_time - start_time

    print("time: ", total_time)

    print("Mejores parámetros para Bayesian:", best_params_bayesian)
    print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")
    return best_params_bayesian

def opt_xgb(space_xgb):
    print(f"\t Optimización de hiperparametros  para XGBRegressor.....\n")
    start_time = time.time()

    trials_xgb = Trials()
    best_params_xgb = fmin(fn=objective_xgb,
                           space=space_xgb,
                           algo=tpe.suggest,
                           max_evals=100,
                           trials=trials_xgb)

    end_time = time.time()
    total_time = end_time - start_time

    print("time: ", total_time)


    print("Mejores parámetros para XGBRegressor:", best_params_xgb)

    print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    return best_params_xgb

def opt_lgb(space_lgbm):
    print(f"\t Optimización de hiperparametros para LightGBM......\n")
    start_time = time.time()

    trials_lgbm = Trials()
    best_params_lgbm = fmin(fn=objective_lgbm, space=space_lgbm, algo=tpe.suggest,
                           max_evals=100, trials=trials_lgbm)
    end_time = time.time()
    total_time = end_time - start_time

    print("time: ", total_time)
    print("Mejores parámetros para LightGBM:", best_params_lgbm)
    print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    return best_params_lgbm



def opt_gb(space_gb):
    print(f"\t Optimización de hiperparametros para Gradient Boosting.....\n")
    start_time = time.time()

    trials_gb = Trials()
    best_params_gb = fmin(fn=objective_gb, space=space_gb, algo=tpe.suggest,
                         max_evals=100, trials=trials_gb)

    end_time = time.time()
    total_time = end_time - start_time

    print("time: ", total_time)

    print("Mejores parámetros para Gradient Boosting:", best_params_gb)
    print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    return best_params_gb



def opt_ab(space_ab):
    print(f"\t Optimización de hiperparametros para AdaBoost.....\n")
    start_time = time.time()


    trials_ab = Trials()
    best_params_ab = fmin(fn=objective_ab, space=space_ab, algo=tpe.suggest,
                         max_evals=100, trials=trials_ab)

    end_time = time.time()
    total_time = end_time - start_time

    print("time: ", total_time)
    print("Mejores parámetros para AdaBoost:", best_params_ab)
    print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    return best_params_ab



def opt_rf(space_rf):
    print(f"\t Optimización de hiperparametros para Random Forest.....\n")
    start_time = time.time()


    trials_rf = Trials()
    best_params_rf = fmin(fn=objective_rf, space=space_rf, algo=tpe.suggest,
                         max_evals=100, trials=trials_rf)

    end_time = time.time()
    total_time = end_time - start_time

    print("time: ", total_time)
    print("Mejores parámetros para Random Forest:", best_params_rf)
    print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    return best_params_rf



def opt_svr(space_svr):
    print(f"\t Optimización de hiperparametros para SVR.....\n")
    start_time = time.time()



    trials_svr = Trials()
    best_params_svr = fmin(fn=objective_svr, space=space_svr, algo=tpe.suggest,
                          max_evals=100, trials=trials_svr)

    end_time = time.time()
    total_time = end_time - start_time

    print("time: ", total_time)
    print("Mejores parámetros para SVR:", best_params_svr)
    print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    return best_params_svr

## Busqueda de hiperparametros

In [18]:
best_params_bayesian = opt_bayesian(space_bayesian)

	 Optimización de hiperparametros para Bayesian.....

RMSE: 0.36410350445822404, R² : 0.5841372396302172, MAE: 0.26415425781034185 ;       
	 PARAMS; alpha_1: 1.751659912070332e-07, alpha_2: 0.00011705005409166187, lambda_1: 3.307179748301921e-05, lambda_2: 4.643248194637293e-07, tol: 1.1836141117268175, max_iter: 600
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.36488396939364465, R² : 0.5816724822313905, MAE: 0.26421082958740005 ;       
	 PARAMS; alpha_1: 0.914225464122633, alpha_2: 5.411588360188509e-06, lambda_1: 0.002180767968282913, lambda_2: 0.013299675765649712, tol: 1.328729436774008, max_iter: 475
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
best_params_xgb = opt_xgb(space_xgb)

	 Optimización de hiperparametros  para XGBRegressor.....

RMSE: 0.3465292974346431, R² : 0.6212493787493414, MAE: 0.23999186807694803 ;        
	 PARAMS; estimator: 915, leaning rate: 0.011862015913734794, gamma: 0.2319611548688738, max_depth: 11, min_child_weight: 8, colsample_b: 0.7390406307276011, subsample: 0.9667296376420299, alpha: 0.5701515496289244, lambda: 0.5207338536578585
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.3510109769489246, R² : 0.6134779632846099, MAE: 0.24919074649760625 ;        
	 PARAMS; estimator: 912, leaning rate: 0.026716632242734534, gamma: 0.7074597844795587, max_depth: 13, min_child_weight: 14, colsample_b: 0.7723920804541826, subsample: 0.9960199905639995, alpha: 0.6495008017830702, lambda: 1.2527188956375193
----------------------------------

In [20]:
best_params_svr = opt_svr(space_svr)

	 Optimización de hiperparametros para SVR.....

RMSE: 0.3694513205202974, R² : 0.5697545370599307, MAE: 0.2567374927922025 ;         
	 PARAMS; C: 2100, epsilon: 0.002290122814485085, degree: 9
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.36950288110970314, R² : 0.5696352719851602, MAE: 0.25677794450748326 ;       
	 PARAMS; C: 1000, epsilon: 0.0019287668869724204, degree: 5
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.3694951316154626, R² : 0.5696552434307158, MAE: 0.25676398575734793 ;        
	 PARAMS; C: 1400, epsilon: 0.001973394979225892, degree: 4
----------------

In [21]:
best_params_rf = opt_rf(space_rf)

	 Optimización de hiperparametros para Random Forest.....

RMSE: 0.34677972451997563, R² : 0.6208263075598272, MAE: 0.23936422244064653 ;       
	 PARAMS; estimator: 2500, max_depth: 21, min_samples_split: 9, min_samples_leaf: 6
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.3464480846877661, R² : 0.6212906097679576, MAE: 0.23895327939193808 ;        
	 PARAMS; estimator: 2400, max_depth: 22, min_samples_split: 6, min_samples_leaf: 5
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.34649217008279676, R² : 0.6211536412584476, MAE: 0.23906353202036668 ;       
	 PARAMS; estimator

In [23]:
best_params_gb = opt_gb(space_gb)

	 Optimización de hiperparametros para Gradient Boosting.....

  0%|                                        | 0/100 [00:00<?, ?trial/s, best loss=?]

RMSE: 0.3340441445647452, R² : 0.6474019891097249, MAE: 0.22544842604757126 ;        
	 PARAMS; estimator: 1250, leaning rate: 0.010737866056293415, max_depth: 21, min_samples_split: 6, min_samples_leaf: 8, subsample: 0.6571367591382394
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.3333281228262586, R² : 0.6484406101536718, MAE: 0.22832368792924118 ;        
	 PARAMS; estimator: 1390, leaning rate: 0.02183836136828919, max_depth: 21, min_samples_split: 4, min_samples_leaf: 10, subsample: 0.5559324071144726
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.3336518400603402, R² :

In [ ]:
best_params_lgbm = opt_lgb(space_lgbm)

	 Optimización de hiperparametros para LightGBM......

RMSE: 0.33753918428103313, R² : 0.6385467973748659, MAE: 0.22963074543331902 ;       
	 PARAMS; estimator: 1178, leaning rate: 0.01640559469869124, max_depth: 49, min_child_weight: 16, num_leaves: 75, colsample_b: 0.08447584439307829
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RMSE: 0.33601737239244445, R² : 0.6415169955765153, MAE: 0.22805113667210208 ;       
	 PARAMS; estimator: 1099, leaning rate: 0.010992747946656476, max_depth: 47, min_child_weight: 18, num_leaves: 76, colsample_b: 0.08762004387936775
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------